## Primary vs Primary + Blooming

In this notebook all the images with the label primary (but not blooming) will be put against images with the label primary and blooming. 

### Imports

In [1]:
from keras.models import Sequential, load_model, Model
from keras.layers import Conv2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam, SGD
from keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau

import numpy as np
import cPickle as pickle
import pandas as pd
import glob
import os
import sys
sys.path.insert(0, '..')
import paths
sys.path.insert(0, '../rainforest')
import data
import data_generators_leo as dgl

Using Theano backend.
Using cuDNN version 5110 on context None
Mapped name None to device cuda0: GeForce GTX 1050 Ti (0000:09:00.0)


### Loading training data and corresponding labels. 

Images = ['image_name', 'primary', 'blooming']

In [8]:
batch_size = 32
size_img = 92
imgs = (size_img,size_img)
#train_csv = data.get_class_data(train=True, label='primary')

train_csv = np.asarray(data.get_class_data(train=True, label='primary')) #everything in onehot
val_csv =  np.asarray(data.get_class_data(train=False, label='primary')) #everything in onehot

train_data = []
for i in train_csv:
    if i[1] == 1:
        appending = [i[0], int(i[1]), int(i[2])]
        train_data.append(appending)
train_csv = np.asarray(train_data)

val_data = []
for i in val_csv:
    if i[1] == 1:
        appending = [i[0], int(i[1]), int(i[2])]
        val_data.append(appending)
val_csv = np.asarray(val_data)


train_data = train_csv[:,0] #image names training
val_data = val_csv[:,0] #image names validation


train_labels = train_csv[:,2:] 
val_labels = val_csv[:,2:]

print train_labels.shape
print val_labels.shape

#train_labels = train_csv[:,11:12] #primary
#val_labels = val_csv[:,2:3]


(26284, 1)
(11229, 1)


In [9]:
dim = len(train_labels)
dimlabel = np.zeros((dim, 2))
for i, j in zip(train_labels, dimlabel):
    j[int(i[0])] = 1
train_labels = dimlabel

dim = len(val_labels)
vallabel = np.zeros((dim, 2))
for i, j in zip(val_labels, vallabel):
    j[int(i[0])] = 1
val_labels = vallabel

In [10]:
print train_labels.shape
print val_labels.shape
print val_data.shape
print train_data.shape

print train_labels[1]
print val_labels[1]
print train_data[1]
print val_data[1]


(26284, 2)
(11229, 2)
(11229,)
(26284,)
[ 1.  0.]
[ 1.  0.]
train_8337
train_19035


#### Generators

In [11]:
train_gen = dgl.get_data(train_data, '../data/train-jpg', train_labels, batch_size=batch_size, img_size=imgs, balance_batches=True, augmentation=True, hflip=True, vflip=True, shift_x=5, shift_y=5, rot_range=10)
val_gen = dgl.get_data(val_data, '../data/train-jpg', val_labels, batch_size=batch_size, img_size=imgs, balance_batches=True)

### Network Structure

In [12]:
model = Sequential()

model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(3, 92, 92)))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Dropout(0.50))
model.add(Dense(2, activation='softmax'))


model.compile(optimizer='Adam', loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 16, 90, 90)        448       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 88, 88)        2320      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 44, 44)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 44, 44)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 32, 42, 42)        4640      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 32, 40, 40)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 20, 20)        0         
__________

### Training Network

In [13]:
steps = len(train_data)/batch_size
steps_val = len(val_data)/batch_size
print steps
print steps_val


csv_logger = CSVLogger('run4_adam.csv')
lr_plateau = ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.5)
checkpoint = ModelCheckpoint(filepath='/home/pieter/projects/MLIP-Brainforest/bloomingtest/OWN/model.{epoch:02d}-{val_loss:.2f}.hdf5',
                             verbose=1, save_best_only=True)

model.fit_generator(train_gen, steps_per_epoch=steps,
                    epochs=50, verbose=1,
                    callbacks=[csv_logger, lr_plateau, checkpoint],
                    validation_data=val_gen, validation_steps=steps_val)

821
350
Epoch 1/50
821/821 [==============================] - 68s - loss: 0.6825 - acc: 0.5592 - val_loss: 0.6836 - val_acc: 0.5723
Epoch 2/50
821/821 [==============================] - 67s - loss: 0.6644 - acc: 0.5792 - val_loss: 0.6876 - val_acc: 0.5595
Epoch 3/50
820/821 [============================>.] - ETA: 0s - loss: 0.6563 - acc: 0.5894
Epoch 00002: reducing learning rate to 0.000500000023749.
Epoch 00002: val_loss did not improve
821/821 [==============================] - 67s - loss: 0.6563 - acc: 0.5894 - val_loss: 0.6871 - val_acc: 0.5810
Epoch 4/50
820/821 [============================>.] - ETA: 0s - loss: 0.6304 - acc: 0.6112
Epoch 00003: reducing learning rate to 0.000250000011874.
Epoch 00003: val_loss did not improve
821/821 [==============================] - 67s - loss: 0.6303 - acc: 0.6112 - val_loss: 0.6954 - val_acc: 0.5658
Epoch 5/50
820/821 [============================>.] - ETA: 0s - loss: 0.6113 - acc: 0.6306
Epoch 00004: reducing learning rate to 0.000125000005

420/821 [==============>...............] - ETA: 24s - loss: 0.5795 - acc: 0.6574

KeyboardInterrupt: 